In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
f = pd.read_csv('/content/drive/MyDrive/Dataset-SA.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
sentences=[]
labels=[]
for item in f['Summary']:
  sentences.append(item)
for item in f['Sentiment']:
  if item=='positive':
    labels.append(1.0)
  elif item=='negative':
    labels.append(0.0)
  elif item=='neutral':
    labels.append(0.5)

trainingSize = int(0.8*(len(sentences)))

In [ ]:
tokenizer = Tokenizer()

In [ ]:
trainingSentences = sentences[:trainingSize]
testingSentences = sentences[trainingSize:]
trainingLabels = labels[:trainingSize]
testingLabels = labels[trainingSize:]

In [ ]:
tokenizer=Tokenizer(oov_token="<OOV>")
trainingSentences = [str(sentence) for sentence in trainingSentences]
tokenizer.fit_on_texts(trainingSentences)
wordIndex=tokenizer.word_index
words=len(wordIndex)
max_length=max(len(i) for i in trainingSentences)

In [ ]:
trainingSequences=tokenizer.texts_to_sequences(trainingSentences)
testingSequences=tokenizer.texts_to_sequences(testingSentences)
from tensorflow.keras.preprocessing.sequence import pad_sequences
paddedSequences=pad_sequences(trainingSequences, maxlen=max_length, padding='pre')
paddedTests=pad_sequences(testingSequences, maxlen=max_length, padding='pre')

In [ ]:
from tensorflow import keras

In [ ]:
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(words+1, 100, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
import numpy as np

In [ ]:
paddedSequences = np.array(paddedSequences)
trainingLabels = np.array(trainingLabels)
paddedTests = np.array(paddedTests)
testingLabels = np.array(testingLabels)


In [ ]:
num_epochs=30
history=model.fit(paddedSequences, trainingLabels, epochs=num_epochs, validation_data=(paddedTests, testingLabels), verbose=2)

Epoch 1/30
5127/5127 - 272s - loss: 0.2629 - accuracy: 0.8698 - val_loss: 0.1769 - val_accuracy: 0.9129 - 272s/epoch - 53ms/step
Epoch 2/30
5127/5127 - 275s - loss: 0.1562 - accuracy: 0.9146 - val_loss: 0.1440 - val_accuracy: 0.9201 - 275s/epoch - 54ms/step
Epoch 3/30
5127/5127 - 267s - loss: 0.1448 - accuracy: 0.9183 - val_loss: 0.1405 - val_accuracy: 0.9212 - 267s/epoch - 52ms/step
Epoch 4/30
5127/5127 - 268s - loss: 0.1354 - accuracy: 0.9209 - val_loss: 0.1315 - val_accuracy: 0.9216 - 268s/epoch - 52ms/step
Epoch 5/30
5127/5127 - 274s - loss: 0.1271 - accuracy: 0.9233 - val_loss: 0.1260 - val_accuracy: 0.9225 - 274s/epoch - 54ms/step
Epoch 6/30
5127/5127 - 274s - loss: 0.1199 - accuracy: 0.9247 - val_loss: 0.1236 - val_accuracy: 0.9247 - 274s/epoch - 54ms/step
Epoch 7/30
5127/5127 - 275s - loss: 0.1145 - accuracy: 0.9264 - val_loss: 0.1486 - val_accuracy: 0.9120 - 275s/epoch - 54ms/step
Epoch 8/30
5127/5127 - 272s - loss: 0.1106 - accuracy: 0.9279 - val_loss: 0.1247 - val_accuracy: 

In [ ]:
sentence=["One of the most iconic locations in Mumbai! A light and sound show will be inaugurated in Feb 2023. A lot of food trails of Mumbai are near this Landmark. Most group tours also include this monument. A ferry to Elephanta Caves departs from here. One can see the lit coastline of Mumbai at sundown!",
          "Stayed in Churchgate area at Sea Green hotel for two nights. Good location for Gateway of India and getting to Elephanta Island. 25 minswalk to the CSMT train station from hotel. There is always a taxi available. Please note the constant noise of car horns beeping. Mumbai is a very noisy city. As a mature female solo traveller I felt safe",
          "Too crowded (like dadar railway station at peak hour) and much of the structure is covered with random objects and tall speakers. No entry inside too. Overall not at all worth it."]
sequences=tokenizer.texts_to_sequences(sentence)
padded=pad_sequences(sequences, maxlen=max_length, padding='pre')
print(model.predict(padded))

1/1 [==============================] - 0s 23ms/step
[[0.99030334]
 [1.        ]
 [0.24249841]]
